In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D,Dense,Dropout,Input,
from tensorflow.keras.layers import Dense,Dropout,MaxPooling2D,Conv2D,GlobalAveragePooling2D,RandomFlip,RandomRotation,RandomBrightness,RandomContrast,RandomZoom,Input,Rescaling,UpSampling2D
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau,TensorBoard
from PIL import Image
from tensorflow.keras import models
import os
import gradio as gr

In [2]:
from tensorflow.keras.datasets import cifar10


In [58]:


# Load the training and test data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()



x_train.shape

(50000, 32, 32, 3)

In [61]:
x_train_gray = tf.image.rgb_to_grayscale(x_train)
x_test_gray = tf.image.rgb_to_grayscale(x_test)

# Convert to float32 before normalization
x_train_gray = tf.cast(x_train_gray, tf.float32) / 255.0
x_test_gray = tf.cast(x_test_gray, tf.float32) / 255.0


In [44]:
def autoencoder(input_shape=(32,32,1)):
    inputs=Input(shape=(input_shape))
    x=Conv2D(32,(3,3),activation='relu',padding='same')(inputs)
    x=MaxPooling2D((2,2),padding='same')(x)
    x=Conv2D(16,(3,3),activation='relu',padding='same')(x)
    encoded=MaxPooling2D((2,2))(x)

    x=UpSampling2D((2,2))(encoded)
    x=Conv2D(32,(3,3),activation='relu',padding='same')(x)
    x=UpSampling2D((2,2))(x)
    x=Conv2D(32,(3,3),activation='relu',padding='same')(x)
    decoded=Conv2D(3,(3,3),activation='sigmoid',padding='same')(x)

    autoencoders=models.Model(inputs,decoded)
    return autoencoders

In [52]:
model=autoencoder()
model.compile(optimizer='adam',loss='mse')
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 32, 32, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 16, 16, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 8, 8, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_10 (UpSampling2D) │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 16, 16, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_11 (UpSampling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 32, 32, 3)      │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,699 (76.95 KB)

 Trainable params: 19,699 (76.95 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
x_train_gray.shape

TensorShape([50000, 32, 32, 1])

In [62]:
model.fit(
    x_train_gray,
    x_train_gray,
    epochs=15,
    validation_data=(x_test_gray,x_test_gray),
    callbacks=[EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True,mode='max'),
               ModelCheckpoint('ecoder/decoder_best_model.keras',monitor='val_loss',verbose=1)],
    verbose=1
)

Epoch 1/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 0.0145
Epoch 1: saving model to ecoder/decoder_best_model.keras
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 256s 163ms/step - loss: 0.0145 - val_loss: 0.0052
Epoch 2/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 0.0049
Epoch 2: saving model to ecoder/decoder_best_model.keras
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 268s 167ms/step - loss: 0.0049 - val_loss: 0.0046
Epoch 3/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - loss: 0.0041
Epoch 3: saving model to ecoder/decoder_best_model.keras
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 274s 175ms/step - loss: 0.0041 - val_loss: 0.0037
Epoch 4/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 0.0036
Epoch 4: saving model to ecoder/decoder_best_model.keras
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 292s 156ms/step - loss: 0.0036 - val_loss: 0.0034


In [55]:
classnames=['gray','colored','mixed']

In [63]:
def prediction(img1):
    img=img1.convert('L').resize((32,32))
    img_array =tf.keras.utils.img_to_array(img)/255.0
    img_batch =np.expand_dims(img_array,axis=0)
    predictions=model.predict(img_batch)[0]
    output=np.clip(predictions,0,1)
    return output

interface=gr.Interface(
    fn=prediction,
    inputs=gr.Image(type='pil'),
    outputs=gr.Image(type='numpy'),
    title ='coloring/decoloring model',
    description='attarch a grayscale image to see the coloring',)
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://226c509d1e54d79aa0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
